In [1]:
require 'nn'
require 'nngraph'
require 'rnn'
require 'cutorch'
require 'cunn'
require 'optim'
LSTM = require 'model.LSTM'

In [26]:
-- model hyper-params
emb_dim = 2
sum_rep_dim = 32
class_types = 2
num_layers = 1
gpu = false -- GPU is not going to help in this small model
-- construct LSTM cell
emb_unit = nn.LookupTable(10, emb_dim)
out_unit = nn.Sequential()
out_unit:add(nn.Linear(sum_rep_dim, class_types))
out_unit:add(nn.LogSoftMax())
lstm_config = {input_dim=emb_dim,
    hidden_dim=sum_rep_dim,
    num_layers=num_layers,
    emb_unit=emb_unit,
    output_unit=out_unit}
master_cell = LSTM.lstm(lstm_config)
criterion = nn.ClassNLLCriterion()
if gpu then 
    master_cell:cuda() 
    criterion:cuda()
end
params, grad_params = master_cell:getParameters()

In [ ]:
-- model hyper-params
emb_dim = 2
sum_rep_dim = 32
class_types = 2
num_layers = 1
gpu = false -- GPU is not going to help in this small model
dropout = true
-- model
model = nn.Sequential()
emb_unit = nn.LookupTable(10, emb_dim)
out_unit = nn.Sequential()
out_unit:add(nn.Linear(sum_rep_dim, class_types))
out_unit:add(nn.LogSoftMax())
lstm_config = {input_dim=emb_dim,
    hidden_dim=sum_rep_dim,
    num_layers=num_layers,
    emb_unit=emb_unit,
    output_unit=out_unit}
rnn = nn.Sequencer(LSTM.lstm(lstm_config))
model:add(rnn)
if dropout then
    lm:add(nn.Sequencer(nn.Dropout(0.5)))
end


In [27]:
-- construct network
seq_len = 10
batch_size = 64
init_states, _init_values = LSTM.init_states(batch_size, num_layers, sum_rep_dim, gpu)
cells = LSTM.duplicate_cells(master_cell, seq_len)

In [28]:
-- generate data
function display(inputs, outputs)
    for b=1,inputs:size(2) do
        print(string.format('Batch: %d', b))
        local s = 0
        for l = 1, inputs:size(1) do
            s = s + inputs[l][b] - 1
            out_text = 'Even'
            if outputs[l][b] == 2 then out_text = 'Odd' end
            print(string.format('input %d, sum %d, output %s', inputs[l][b] - 1, s, out_text))
        end
    end
end

function gendata(batch_size, len, num, cuda)
    local D = {}
    local Y = {}
    for n=1,num do
        D[n] = torch.Tensor(len, batch_size)
        Y[n] = torch.Tensor(len, batch_size)
        for b=1,batch_size do
            for l=1,len do
                D[n][l][b] = torch.uniform(1,11)
            end
        end
        D[n]:floor()
        for b=1,batch_size do
            local s = 0
            for l=1,len do
                s = s + D[n][l][b] - 1
                Y[n][l][b] = (s % 2) + 1
            end
        end
        if cuda then
            D[n] = D[n]:cuda()
            Y[n] = Y[n]:cuda()
        end
    end
    return D, Y
end
D, Y = gendata(batch_size, seq_len, 600, gpu)

In [29]:
for n = 1, seq_len do
    cells[n]:training()
end
-- for optim
local n=1
function feval(x)
    -- reset
    if x ~= params then
        params:copy(x)
    end
    grad_params:zero()
    -- get data
    local data = D[n]
    local labels = Y[n]
    
    -- forward
    local rstates = {[0]=init_states}
    local outputs = {}
    local loss = 0
    for t=1,seq_len do
        local cell_output = cells[t]:forward{data[t], unpack(rstates[t-1])}
        if num_layers == 1 then 
            cell_output[1] = {cell_output[1]}
            cell_output[2] = {cell_output[2]}
        end
        rstates[t] = {cell_output[1], cell_output[2]}
        outputs[t] = cell_output[3]
        -- this will average loss for each batch
        loss = loss + criterion:forward(outputs[t], labels[t])
    end
    loss = loss / seq_len
    -- backward
    local zero_grads, _ = LSTM.init_states(batch_size, num_layers, sum_rep_dim, gpu)
    local grads = {[seq_len]=zero_grads}
    for t = seq_len,1,-1 do
        local grad_output = criterion:backward(outputs[t], labels[t])
        table.insert(grads[t], grad_output)
        if num_layers == 1 then
            grads[t][1] = grads[t][1][1]
            grads[t][2] = grads[t][2][1]
        end
        local grad_cell = cells[t]:backward(
            {data[t], unpack(rstates[t-1])},
            grads[t])
        grads[t-1] = {}
        grads[t-1][1] = grad_cell[2]
        grads[t-1][2] = grad_cell[3]
    end
    grad_params:div(seq_len)
    -- update data to the next batch
    n = n + 1
    if n > #D then n = 1 end
    -- in case, that data is continuous between batch
    -- init_states = rstates[seq_len]
    return loss, grad_params
end
sgd_params = {
   learningRate = 1e-0,
   -- learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}


In [30]:
local epochs = 2
local nt = torch.getnumthreads()
torch.setnumthreads(1)
for j = 1, epochs do
    local cur_loss = 0
    local timer = torch.Timer() 
    for i = 1, 600 do
        _, fs = optim.sgd(feval, params, sgd_params)
        cur_loss = cur_loss + fs[1]
    end
    local ts = timer:time().real
    cur_loss = cur_loss / 600
    print(string.format('[%ss] %f', ts, cur_loss))
    if cur_loss < 0.03 then break end
end
torch.setnumthreads(nt)

[5.6528389453888s] 0.673963	


[5.3023719787598s] 0.144206	


[5.2973129749298s] 0.001075	


In [31]:
local rstates = {[0]=init_states}
local outputs = {}
local loss = 0
local data = D[1]
local labels = Y[1]
for t=1,seq_len do
    local cell_output = cells[t]:forward{data[t], unpack(rstates[t-1])}
    if num_layers == 1 then 
        cell_output[1] = {cell_output[1]}
        cell_output[2] = {cell_output[2]}
    end
    rstates[t] = {cell_output[1], cell_output[2]}
    outputs[t] = cell_output[3]
    -- this will average loss for each batch
    loss = loss + criterion:forward(outputs[t], labels[t])
end
local predicts = {}
for t = 1, seq_len do
    predicts[t] = {}
    for b = 1, batch_size do
        local _,d = torch.max(outputs[t][b],1)
        predicts[t][b] = d[1]
    end
end
display(data, predicts)

Batch: 1	
input 8, sum 8, output Even	
input 7, sum 15, output Odd	
input 2, sum 17, output Odd	
input 7, sum 24, output Even	
input 2, sum 26, output Even	
input 2, sum 28, output Even	
input 0, sum 28, output Even	
input 2, sum 30, output Even	
input 2, sum 32, output Even	
input 1, sum 33, output Odd	
Batch: 2	
input 3, sum 3, output Odd	
input 0, sum 3, output Odd	
input 1, sum 4, output Even	


input 0, sum 4, output Even	
input 8, sum 12, output Even	
input 9, sum 21, output Odd	
input 1, sum 22, output Even	
input 9, sum 31, output Odd	
input 4, sum 35, output Odd	
input 5, sum 40, output Even	
Batch: 3	
input 4, sum 4, output Even	
input 4, sum 8, output Even	
input 7, sum 15, output Odd	
input 2, sum 17, output Odd	
input 8, sum 25, output Odd	
input 7, sum 32, output Even	
input 9, sum 41, output Odd	
input 7, sum 48, output Even	
input 1, sum 49, output Odd	
input 8, sum 57, output Odd	
Batch: 4	
input 3, sum 3, output Odd	
input 8, sum 11, output Odd	
input 6, sum 17, output Odd	


input 1, sum 18, output Even	
input 6, sum 24, output Even	
input 1, sum 25, output Odd	
input 1, sum 26, output Even	
input 8, sum 34, output Even	
input 9, sum 43, output Odd	
input 2, sum 45, output Odd	
Batch: 5	
input 2, sum 2, output Even	
input 3, sum 5, output Odd	
input 6, sum 11, output Odd	
input 3, sum 14, output Even	
input 4, sum 18, output Even	
input 1, sum 19, output Odd	
input 7, sum 26, output Even	
input 2, sum 28, output Even	
input 1, sum 29, output Odd	
input 9, sum 38, output Even	
Batch: 6	
input 8, sum 8, output Even	
input 5, sum 13, output Odd	
input 4, sum 17, output Odd	
input 0, sum 17, output Odd	


input 5, sum 22, output Even	
input 1, sum 23, output Odd	
input 1, sum 24, output Even	
input 8, sum 32, output Even	
input 1, sum 33, output Odd	
input 0, sum 33, output Odd	
Batch: 7	
input 4, sum 4, output Even	
input 1, sum 5, output Odd	
input 4, sum 9, output Odd	
input 2, sum 11, output Odd	
input 7, sum 18, output Even	
input 0, sum 18, output Even	
input 9, sum 27, output Odd	
input 3, sum 30, output Even	
input 0, sum 30, output Even	
input 8, sum 38, output Even	
Batch: 8	
input 7, sum 7, output Odd	
input 8, sum 15, output Odd	
input 9, sum 24, output Even	
input 0, sum 24, output Even	
input 2, sum 26, output Even	


input 6, sum 32, output Even	
input 4, sum 36, output Even	
input 5, sum 41, output Odd	
input 3, sum 44, output Even	
input 3, sum 47, output Odd	
Batch: 9	
input 4, sum 4, output Even	
input 3, sum 7, output Odd	
input 4, sum 11, output Odd	
input 7, sum 18, output Even	
input 9, sum 27, output Odd	
input 7, sum 34, output Even	
input 1, sum 35, output Odd	
input 3, sum 38, output Even	
input 2, sum 40, output Even	
input 5, sum 45, output Odd	
Batch: 10	
input 2, sum 2, output Even	
input 3, sum 5, output Odd	
input 5, sum 10, output Even	
input 8, sum 18, output Even	
input 4, sum 22, output Even	


...

input 9, sum 49, output Odd	
input 1, sum 50, output Even	
Batch: 55	
input 4, sum 4, output Even	
input 5, sum 9, output Odd	
input 0, sum 9, output Odd	
input 2, sum 11, output Odd	
input 9, sum 20, output Even	
input 2, sum 22, output Even	
input 9, sum 31, output Odd	
input 2, sum 33, output Odd	
input 7, sum 40, output Even	
input 5, sum 45, output Odd	
Batch: 56	
input 1, sum 1, output Odd	
input 2, sum 3, output Odd	
input 3, sum 6, output Even	
input 6, sum 12, output Even	
input 9, sum 21, output Odd	
input 4, sum 25, output Odd	
input 9, sum 34, output Even	
input 2, sum 36, output Even	
input 9, sum 45, output Odd	
input 8, sum 53, output Odd	


Batch: 57	
input 8, sum 8, output Even	
input 2, sum 10, output Even	
input 6, sum 16, output Even	
input 2, sum 18, output Even	
input 3, sum 21, output Odd	
input 3, sum 24, output Even	
input 2, sum 26, output Even	
input 2, sum 28, output Even	
input 0, sum 28, output Even	
input 9, sum 37, output Odd	
Batch: 58	
input 1, sum 1, output Odd	
input 7, sum 8, output Even	
input 0, sum 8, output Even	
input 6, sum 14, output Even	
input 1, sum 15, output Odd	
input 2, sum 17, output Odd	
input 9, sum 26, output Even	
input 2, sum 28, output Even	
input 2, sum 30, output Even	
input 4, sum 34, output Even	
Batch: 59	


input 5, sum 5, output Odd	
input 2, sum 7, output Odd	
input 9, sum 16, output Even	
input 7, sum 23, output Odd	
input 7, sum 30, output Even	
input 8, sum 38, output Even	
input 3, sum 41, output Odd	
input 1, sum 42, output Even	
input 4, sum 46, output Even	
input 0, sum 46, output Even	
Batch: 60	
input 2, sum 2, output Even	
input 4, sum 6, output Even	
input 7, sum 13, output Odd	
input 4, sum 17, output Odd	
input 5, sum 22, output Even	
input 3, sum 25, output Odd	
input 6, sum 31, output Odd	
input 8, sum 39, output Odd	
input 6, sum 45, output Odd	
input 6, sum 51, output Odd	
Batch: 61	
input 9, sum 9, output Odd	
input 6, sum 15, output Odd	


input 7, sum 22, output Even	
input 7, sum 29, output Odd	
input 3, sum 32, output Even	
input 7, sum 39, output Odd	
input 6, sum 45, output Odd	
input 1, sum 46, output Even	
input 2, sum 48, output Even	
input 6, sum 54, output Even	
Batch: 62	
input 1, sum 1, output Odd	
input 6, sum 7, output Odd	
input 2, sum 9, output Odd	
input 9, sum 18, output Even	
input 3, sum 21, output Odd	
input 2, sum 23, output Odd	
input 8, sum 31, output Odd	
input 9, sum 40, output Even	
input 6, sum 46, output Even	
input 7, sum 53, output Odd	
Batch: 63	
input 4, sum 4, output Even	
input 9, sum 13, output Odd	
input 4, sum 17, output Odd	
input 4, sum 21, output Odd	


input 4, sum 25, output Odd	
input 9, sum 34, output Even	
input 8, sum 42, output Even	
input 4, sum 46, output Even	
input 5, sum 51, output Odd	
input 1, sum 52, output Even	
Batch: 64	
input 1, sum 1, output Odd	
input 5, sum 6, output Even	
input 0, sum 6, output Even	
input 1, sum 7, output Odd	
input 6, sum 13, output Odd	
input 8, sum 21, output Odd	
input 1, sum 22, output Even	
input 5, sum 27, output Odd	
input 5, sum 32, output Even	
input 3, sum 35, output Odd	


In [7]:
cutorch.getMemoryUsage(1)

4123680768	4294770688	
